In [1]:
# Import relevant libraries
import pandas as pd

In [5]:
def DimPlayer():
    '''This function cleans and prepares the Player information file for use with fact tables'''
    data = pd.read_excel("Raw Files/PlayerDetails.xlsx") # import original file
    data = data.drop_duplicates(subset = ['Player']) # same player, different season
    data['Player'] = data['Player'].str.replace(',','')
    
    l1 = []
    for ht in data['HT']:
        if ht.split('-')[0] == '':
            l1.append(0)
        else:
            l1.append(int(ht.split('-')[0])*12 + int(ht.split('-')[1]))
    data['HT'] = l1
    
    data['WT'] = data['WT'].str.replace('-','0')
    data['WT'] = pd.to_numeric(data['WT'], errors = 'coerce') # convert to numeric
    data['Draft Status'] = data['Draft Status'].str.replace(',',' -')
    data['Nationality'] = data['Nationality'].str.replace(',',' &')
    
    # specifying player type
    l2 = []
    for x in data['Nationality']:
        if 'Australia' in x.split(' & ') or 'New Zealand' in x.split(' & '):
            l2.append('Non-restricted')
        else:
            l2.append('Restricted')
    data['Player Type'] = l2
    
    data['PlayerID'] = range(1, len(data)+1)
    with open('DimPlayer.csv', 'w') as f:
        data.to_csv(f, line_terminator = '\n', index = False) # convert to CSV file
        f.flush() # data written to file immediately
    
DimPlayer()

In [20]:
def TotalsCSV():
    '''This function cleans and prepares the Total statistics for uploading to the warehouse'''
    data = pd.read_excel("Totals.xls") # import original file    
    data['MIN'] = data['MIN'].str.replace(',','')
    data['Player'] = data['Player'].str.replace(',','')
    data['MIN'] = pd.to_numeric(data['MIN'], errors = 'coerce') # convert to numeric
    with open('TotalsCSV.csv', 'w') as f:
        data.to_csv(f, line_terminator = '\n', index = False) # convert to CSV file
        f.flush() # data written to file immediately

TotalsCSV()

In [13]:
def TotalsFact():
    # Open CSV files to be matched
    totals_df = pd.read_csv('TotalsCSV.csv')
    team_df = pd.read_csv('DimTeam.csv')
    position_df = pd.read_csv('DimPosition.csv')
    player_df = pd.read_csv('DimPlayer.csv')

    # replacing Team with ID
    final_df = pd.merge(totals_df, team_df, on = 'Team', how = 'inner').drop(['Team', 'Name'], axis = 1)
    
    # replacing Position with ID
    final_df = pd.merge(final_df, position_df, on = 'Position', how = 'inner').drop(['Position', 'Name'], axis = 1)

    # replacing Player with ID
    final_df = pd.merge(final_df, player_df, on = 'Player', how = 'left').drop(['Player', 'HT', 'WT', 'Birth City',
                                                                                 'Draft Status', 'Nationality'], axis = 1)
    
    final_df['PlayerID'] = final_df['PlayerID'].fillna(0) # replace nulls with 0
    final_df['PlayerID'] = final_df['PlayerID'].astype(int) # convert to integer
    
    final_df.index += 1 # start index from 1
    with open('TotalsFact.csv', 'w') as f:
        final_df.to_csv(f, line_terminator = '\n', header = False) # convert to CSV file
        f.flush() # data written to file immediately

TotalsFact()

In [20]:
def AveragesCSV():
    '''This function cleans and prepares the Average statistics for uploading to the warehouse'''
    data = pd.read_excel("Averages.xls") # import original file
    data['Player'] = data['Player'].str.replace(',','')
    with open('AveragesCSV.csv', 'w') as f:
        data.to_csv(f, line_terminator = '\n', index = False) # convert to CSV file
        f.flush() # data written to file immediately
    
AveragesCSV()

In [14]:
def AveragesFact():
    # Open CSV files to be matched
    averages_df = pd.read_csv('AveragesCSV.csv')
    team_df = pd.read_csv('DimTeam.csv')
    position_df = pd.read_csv('DimPosition.csv')
    player_df = pd.read_csv('DimPlayer.csv')

    # replacing Team with ID
    final_df = pd.merge(averages_df, team_df, on = 'Team', how = 'inner').drop(['Team', 'Name'], axis = 1)
    
    # replacing Position with ID
    final_df = pd.merge(final_df, position_df, on = 'Position', how = 'inner').drop(['Position', 'Name'], axis = 1)

    # replacing Player with ID
    final_df = pd.merge(final_df, player_df, on = 'Player', how = 'left').drop(['Player', 'HT', 'WT', 'Birth City',
                                                                                 'Draft Status', 'Nationality'], axis = 1)
    
    final_df['PlayerID'] = final_df['PlayerID'].fillna(0) # replace nulls with 0
    final_df['PlayerID'] = final_df['PlayerID'].astype(int) # convert to integer
    
    final_df.index += 1 # start index from 1
    with open('AveragesFact.csv', 'w') as f:
        final_df.to_csv(f, line_terminator = '\n', header = False) # convert to CSV file
        f.flush() # data written to file immediately

AveragesFact()

In [24]:
def AdvancedStatsCSV():
    '''This function cleans and prepares the Advanced statistics for uploading to the warehouse'''
    data = pd.read_excel('AdvancedStats.xls') # import original file
    data['Player'] = data['Player'].str.replace(',','')
    with open('AdvancedStatsCSV.csv', 'w') as f:
        data.to_csv(f, line_terminator = '\n', index = False) # convert to CSV file
        f.flush() # data written to file immediately
    
AdvancedStatsCSV()

In [15]:
def AdvancedStatsFact():
    # Open CSV files to be matched
    AdvancedStats_df = pd.read_csv('AdvancedStatsCSV.csv')
    team_df = pd.read_csv('DimTeam.csv')
    position_df = pd.read_csv('DimPosition.csv')
    player_df = pd.read_csv('DimPlayer.csv')

    # replacing Team with ID
    final_df = pd.merge(AdvancedStats_df, team_df, on = 'Team', how = 'inner').drop(['Team', 'Name'], axis = 1)
    
    # replacing Position with ID
    final_df = pd.merge(final_df, position_df, on = 'Position', how = 'inner').drop(['Position', 'Name'], axis = 1)

    # replacing Player with ID
    final_df = pd.merge(final_df, player_df, on = 'Player', how = 'left').drop(['Player', 'HT', 'WT', 'Birth City',
                                                                                 'Draft Status', 'Nationality'], axis = 1)
    
    final_df['PlayerID'] = final_df['PlayerID'].fillna(0) # replace nulls with 0
    final_df['PlayerID'] = final_df['PlayerID'].astype(int) # convert to integer
    
    final_df.index += 1 # start index from 1
    with open('AdvancedStatsFact.csv', 'w') as f:
        final_df.to_csv(f, line_terminator = '\n', header = False) # convert to CSV file
        f.flush() # data written to file immediately

AdvancedStatsFact()

In [2]:
def PerMinuteCSV():
    '''This function cleans and prepares the Per Minute statistics for uploading to the warehouse'''
    data = pd.read_excel("PerMinuteStats.xls") # import original file    
    data['Player'] = data['Player'].str.replace(',','')
    with open('PerMinuteCSV.csv', 'w') as f:
        data.to_csv(f, line_terminator = '\n', index = False) # convert to CSV file
        f.flush() # data written to file immediately

PerMinuteCSV()

In [3]:
def PerMinuteFact():
    # Open CSV files to be matched
    perminute_df = pd.read_csv('PerMinuteCSV.csv')
    team_df = pd.read_csv('DimTeam.csv')
    position_df = pd.read_csv('DimPosition.csv')
    player_df = pd.read_csv('DimPlayer.csv')

    # replacing Team with ID
    final_df = pd.merge(perminute_df, team_df, on = 'Team', how = 'inner').drop(['Team', 'Name'], axis = 1)
    
    # replacing Position with ID
    final_df = pd.merge(final_df, position_df, on = 'Position', how = 'inner').drop(['Position', 'Name'], axis = 1)

    # replacing Player with ID
    final_df = pd.merge(final_df, player_df, on = 'Player', how = 'left').drop(['Player', 'HT', 'WT', 'Birth City',
                                                                                 'Draft Status', 'Nationality'], axis = 1)
    
    final_df['PlayerID'] = final_df['PlayerID'].fillna(0) # replace nulls with 0
    final_df['PlayerID'] = final_df['PlayerID'].astype(int) # convert to integer
    
    final_df.index += 1 # start index from 1
    with open('PerMinuteFact.csv', 'w') as f:
        final_df.to_csv(f, line_terminator = '\n', header = False) # convert to CSV file
        f.flush() # data written to file immediately

PerMinuteFact()

In [3]:
def DimPlayer2():
    '''This function cleans and prepares the Player information file for uploading to the warehouse'''
    data = pd.read_csv("DimPlayer.csv") # import original file
    with open('DimPlayer.csv', 'w') as f:
        data.to_csv(f, line_terminator = '\n', index = False, header = False) # convert to CSV file
        f.flush() # data written to file immediately
    
DimPlayer2()